# End-to-End Knowledge Graph Construction Demo

This notebook demonstrates a complete multi-agent workflow for knowledge graph construction using Google's Agent Development Kit (ADK). The workflow includes:

1. **User Intent Capture** - Understanding what kind of knowledge graph the user wants
2. **File Suggestion** - Identifying relevant files for the knowledge graph
3. **Schema Proposal (Structured)** - Creating construction rules for structured data
4. **Schema Proposal (Unstructured)** - Extracting entities and facts from text
5. **Knowledge Graph Construction** - Building the actual graph from the approved plans

Each step uses specialized AI agents that collaborate to build a comprehensive knowledge graph.

## Setup and Imports

In [ ]:
# Import necessary libraries
import os
import json
from pathlib import Path
from itertools import islice
from typing import Dict, Any, List, Optional

# Google ADK imports
from google.adk.agents import Agent, LlmAgent, LoopAgent
from google.adk.models.lite_llm import LiteLlm
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.adk.tools import ToolContext, agent_tool
from google.adk.agents.callback_context import CallbackContext
from google.adk.agents.invocation_context import InvocationContext
from google.adk.agents.base_agent import BaseAgent
from google.adk.events import Event, EventActions
from google.genai import types

import warnings
warnings.filterwarnings("ignore")

import logging
logging.basicConfig(level=logging.CRITICAL)

print("Libraries imported successfully!")

## Helper Functions and Utilities

In [ ]:
# Helper functions from helper.py
def get_neo4j_import_dir():
    """Gets the neo4j import directory"""
    return "data/"

class AgentCaller:
    """A simple wrapper class for interacting with an ADK agent."""
    
    def __init__(self, agent: Agent, runner: Runner, user_id: str, session_id: str):
        self.agent = agent
        self.runner = runner
        self.user_id = user_id
        self.session_id = session_id

    async def get_session(self):
        return await self.runner.session_service.get_session(
            app_name=self.runner.app_name, 
            user_id=self.user_id, 
            session_id=self.session_id
        )

    async def call(self, query: str, verbose: bool = False):
        print(f"\n>>> User Query: {query}")
        content = types.Content(role='user', parts=[types.Part(text=query)])
        final_response_text = "Agent did not produce a final response."
        
        async for event in self.runner.run_async(
            user_id=self.user_id, 
            session_id=self.session_id, 
            new_message=content
        ):
            if verbose:
                print(f" [Event] Author: {event.author}, Type: {type(event).__name__}, Final: {event.is_final_response()}, Content: {event.content}")
            
            if event.is_final_response():
                if event.content and event.content.parts:
                    final_response_text = event.content.parts[0].text
                elif event.actions and event.actions.escalate:
                    final_response_text = f"Agent escalated: {event.error_message or 'No specific message.'}"
                
                if event.author == self.agent.name:
                    break
        
        print(f"<<< Agent Response: {final_response_text}")
        return final_response_text

async def make_agent_caller(agent: Agent, initial_state: Optional[Dict[str, Any]] = None) -> AgentCaller:
    """Create and return an AgentCaller instance for the given agent."""
    if initial_state is None:
        initial_state = {}
    
    session_service = InMemorySessionService()
    app_name = agent.name + "_app"
    user_id = agent.name + "_user"
    session_id = agent.name + "_session_01"
    
    await session_service.create_session(
        app_name=app_name,
        user_id=user_id,
        session_id=session_id,
        state=initial_state
    )
    
    runner = Runner(
        agent=agent,
        app_name=app_name,
        session_service=session_service
    )
    
    return AgentCaller(agent, runner, user_id, session_id)

print("Helper functions defined!")

## Neo4j Integration

In [ ]:
# Neo4j integration from neo4j_for_adk.py
def tool_success(key: str, result: Any) -> Dict[str, Any]:
    """Convenience function to return a success result."""
    return {
        'status': 'success',
        key: result
    }

def tool_error(message: str) -> Dict[str, Any]:
    """Convenience function to return an error result."""
    return {
        'status': 'error',
        'error_message': message
    }

# Mock Neo4j class for demo purposes
class MockNeo4jForADK:
    """Mock Neo4j wrapper for demo purposes."""
    
    def send_query(self, cypher_query, parameters=None) -> Dict[str, Any]:
        """Mock query execution that returns success for demo."""
        print(f"Executing Cypher: {cypher_query[:100]}...")
        if parameters:
            print(f"Parameters: {parameters}")
        
        # Return mock success for demo
        return tool_success("query_result", [{"message": "Query executed successfully"}])

# Initialize mock graphdb
graphdb = MockNeo4jForADK()

print("Neo4j integration initialized (mock mode for demo)!")

## LLM Setup

In [ ]:
# LLM configuration
MODEL_GPT = "openai/gpt-4o"
API_KEY = "your-openai-api-key-here"  # Replace with your actual API key

llm = LiteLlm(model=MODEL_GPT)

# Test LLM connection
try:
    response = llm.llm_client.completion(
        model=llm.model,
        api_key=API_KEY,
        messages=[{"role": "user", "content": "Are you ready?"}],
        tools=[]
    )
    print("LLM is ready for use!")
    print(f"Test response: {response.choices[0].message.content}")
except Exception as e:
    print(f"LLM connection failed: {e}")
    print("Please update the API_KEY variable with your OpenAI API key")

## Step 1: User Intent Capture Agent

In [ ]:
# User Intent Agent - captures what kind of knowledge graph the user wants

# Constants
PERCEIVED_USER_GOAL = "perceived_user_goal"
APPROVED_USER_GOAL = "approved_user_goal"

# Tool definitions
def set_perceived_user_goal(kind_of_graph: str, graph_description: str, tool_context: ToolContext):
    """Sets the perceived user's goal, including the kind of graph and its description."""
    user_goal_data = {"kind_of_graph": kind_of_graph, "description": graph_description}
    tool_context.state[PERCEIVED_USER_GOAL] = user_goal_data
    return tool_success(PERCEIVED_USER_GOAL, user_goal_data)

def approve_perceived_user_goal(dummy: str = "", tool_context: ToolContext = None):
    """Upon approval from user, will record the perceived user goal as the approved user goal."""
    if tool_context is None:
        return tool_error("Tool context is required.")
    
    if PERCEIVED_USER_GOAL not in tool_context.state:
        return tool_error("perceived_user_goal not set. Set perceived user goal first, or ask clarifying questions if you are unsure.")
    
    tool_context.state[APPROVED_USER_GOAL] = tool_context.state[PERCEIVED_USER_GOAL]
    return tool_success(APPROVED_USER_GOAL, tool_context.state[APPROVED_USER_GOAL])

def get_approved_user_goal(tool_context: ToolContext):
    """Returns the approved user goal from the tool context state, if it exists."""
    if APPROVED_USER_GOAL not in tool_context.state:
        return tool_error("No approved user goal found. Approve a user goal first.")
    return tool_success(APPROVED_USER_GOAL, tool_context.state[APPROVED_USER_GOAL])

# Agent instruction
user_intent_instruction = """
You are an expert at knowledge graph use cases. 
Your primary goal is to help the user come up with a knowledge graph use case.

If the user is unsure what to do, make some suggestions based on classic use cases like:
- social network involving friends, family, or professional relationships
- logistics network with suppliers, customers, and partners
- recommendation system with customers, products, and purchase patterns
- fraud detection over multiple accounts with suspicious patterns of transactions
- pop-culture graphs with movies, books, or music

A user goal has two components:
- kind_of_graph: at most 3 words describing the graph, for example "social network" or "USA freight logistics"
- description: a few sentences about the intention of the graph, for example "A dynamic routing and delivery system for cargo." or "Analysis of product dependencies and supplier alternatives."

Think carefully and collaborate with the user:
1. Understand the user's goal, which is a kind_of_graph with description
2. Ask clarifying questions as needed
3. When you think you understand their goal, use the 'set_perceived_user_goal' tool to record your perception
4. Present the perceived user goal to the user for confirmation
5. If the user agrees, use the 'approve_perceived_user_goal' tool to approve the user goal. This will save the goal in state under the 'approved_user_goal' key.
"""

# Create the agent
user_intent_agent = Agent(
    name="user_intent_agent_v1",
    model=llm,
    description="Helps the user ideate on a knowledge graph use case.",
    instruction=user_intent_instruction,
    tools=[set_perceived_user_goal, approve_perceived_user_goal]
)

print("User Intent Agent created!")

## Step 2: File Suggestion Agent

In [ ]:
# File Suggestion Agent - helps select relevant files for the knowledge graph

# Constants
ALL_AVAILABLE_FILES = "all_available_files"
SUGGESTED_FILES = "suggested_files"
APPROVED_FILES = "approved_files"

# Tool definitions
def list_available_files(tool_context: ToolContext) -> dict:
    """Lists files available for knowledge graph construction."""
    # Mock file list for demo
    file_names = [
        'products.csv', 
        'suppliers.csv', 
        'parts.csv', 
        'part_supplier_mapping.csv', 
        'assemblies.csv',
        'product_reviews/gothenburg_table_reviews.md',
        'product_reviews/stockholm_chair_reviews.md'
    ]
    
    tool_context.state[ALL_AVAILABLE_FILES] = file_names
    return tool_success(ALL_AVAILABLE_FILES, file_names)

def sample_file(file_path: str, tool_context: ToolContext) -> dict:
    """Samples a file by reading its content as text."""
    # Mock file content for demo
    mock_content = {
        'products.csv': 'product_id,product_name,price,description\nP001,Gothenburg Table,299,Modern dining table\nP002,Stockholm Chair,89,Comfortable office chair',
        'suppliers.csv': 'supplier_id,name,specialty,city,country\nS001,Nordic Wood,Wood Products,Stockholm,Sweden\nS002,Steel Co,Metal Parts,Oslo,Norway',
        'parts.csv': 'part_id,part_name,quantity,assembly_id\nPT001,Table Top,1,A001\nPT002,Table Leg,4,A001',
        'assemblies.csv': 'assembly_id,assembly_name,quantity,product_id\nA001,Main Assembly,1,P001\nA002,Base Assembly,1,P001',
        'part_supplier_mapping.csv': 'part_id,supplier_id,lead_time_days,unit_cost\nPT001,S001,14,45.00\nPT002,S001,7,12.50'
    }
    
    content = mock_content.get(file_path, f"Mock content for {file_path}")
    return tool_success("content", content)

def set_suggested_files(suggest_files: List[str], tool_context: ToolContext) -> Dict[str, Any]:
    """Set the suggested files to be used for data import."""
    tool_context.state[SUGGESTED_FILES] = suggest_files
    return tool_success(SUGGESTED_FILES, suggest_files)

def get_suggested_files(tool_context: ToolContext) -> Dict[str, Any]:
    """Get the files to be used for data import."""
    return tool_success(SUGGESTED_FILES, tool_context.state.get(SUGGESTED_FILES, []))

def approve_suggested_files(dummy: str = "", tool_context: ToolContext = None) -> Dict[str, Any]:
    """Approves the suggested files for further processing."""
    if tool_context is None:
        return tool_error("Tool context is required.")
    
    if SUGGESTED_FILES not in tool_context.state:
        return tool_error("Current files have not been set. Take no action other than to inform user.")
    
    tool_context.state[APPROVED_FILES] = tool_context.state[SUGGESTED_FILES]
    return tool_success(APPROVED_FILES, tool_context.state[APPROVED_FILES])

def get_approved_files(tool_context: ToolContext) -> dict:
    """Returns the approved files from the tool context state."""
    if APPROVED_FILES not in tool_context.state:
        return tool_error("No approved files found. Approve suggested files first.")
    return tool_success(APPROVED_FILES, tool_context.state[APPROVED_FILES])

# Agent instruction
file_suggestion_instruction = """
You are a constructive critic AI reviewing a list of files. Your goal is to suggest relevant files
for constructing a knowledge graph.

**Task:**
Review the file list for relevance to the kind of graph and description specified in the approved user goal. 

For any file that you're not sure about, use the 'sample_file' tool to get 
a better understanding of the file contents. 

Only consider structured data files like CSV or JSON.

Prepare for the task:
- use the 'get_approved_user_goal' tool to get the approved user goal

Think carefully, repeating these steps until finished:
1. list available files using the 'list_available_files' tool
2. evaluate the relevance of each file, then record the list of suggested files using the 'set_suggested_files' tool
3. use the 'get_suggested_files' tool to get the list of suggested files
4. ask the user to approve the set of suggested files
5. If the user has feedback, go back to step 1 with that feedback in mind
6. If approved, use the 'approve_suggested_files' tool to record the approval
"""

# Create the agent
file_suggestion_agent = Agent(
    name="file_suggestion_agent_v1",
    model=llm,
    description="Helps the user select files to import.",
    instruction=file_suggestion_instruction,
    tools=[get_approved_user_goal, list_available_files, sample_file, 
           set_suggested_files, get_suggested_files, approve_suggested_files]
)

print("File Suggestion Agent created!")

## Step 3: Schema Proposal Agent (Structured Data)

In [ ]:
# Schema Proposal Agent for Structured Data

# Constants
PROPOSED_CONSTRUCTION_PLAN = "proposed_construction_plan"
APPROVED_CONSTRUCTION_PLAN = "approved_construction_plan"
NODE_CONSTRUCTION = "node_construction"
RELATIONSHIP_CONSTRUCTION = "relationship_construction"
SEARCH_RESULTS = "search_results"

# Tool definitions
def search_file(file_path: str, query: str) -> dict:
    """Searches any text file for lines containing the given query string."""
    # Mock search for demo
    mock_results = {
        "metadata": {
            "path": file_path,
            "query": query,
            "lines_found": 1 if query in file_path else 0
        },
        "matching_lines": [
            {"line_number": 1, "content": f"Mock result for {query} in {file_path}"}
        ] if query in file_path else []
    }
    return tool_success(SEARCH_RESULTS, mock_results)

def propose_node_construction(approved_file: str, proposed_label: str, 
                            unique_column_name: str, proposed_properties: list[str], 
                            tool_context: ToolContext) -> dict:
    """Propose a node construction for an approved file."""
    construction_plan = tool_context.state.get(PROPOSED_CONSTRUCTION_PLAN, {})
    node_construction_rule = {
        "construction_type": "node",
        "source_file": approved_file,
        "label": proposed_label,
        "unique_column_name": unique_column_name,
        "properties": proposed_properties
    }
    construction_plan[proposed_label] = node_construction_rule
    tool_context.state[PROPOSED_CONSTRUCTION_PLAN] = construction_plan
    return tool_success(NODE_CONSTRUCTION, node_construction_rule)

def propose_relationship_construction(approved_file: str, proposed_relationship_type: str,
                                    from_node_label: str, from_node_column: str,
                                    to_node_label: str, to_node_column: str,
                                    proposed_properties: list[str],
                                    tool_context: ToolContext) -> dict:
    """Propose a relationship construction for an approved file."""
    construction_plan = tool_context.state.get(PROPOSED_CONSTRUCTION_PLAN, {})
    relationship_construction_rule = {
        "construction_type": "relationship",
        "source_file": approved_file,
        "relationship_type": proposed_relationship_type,
        "from_node_label": from_node_label,
        "from_node_column": from_node_column,
        "to_node_label": to_node_label,
        "to_node_column": to_node_column,
        "properties": proposed_properties
    }
    construction_plan[proposed_relationship_type] = relationship_construction_rule
    tool_context.state[PROPOSED_CONSTRUCTION_PLAN] = construction_plan
    return tool_success(RELATIONSHIP_CONSTRUCTION, relationship_construction_rule)

def get_proposed_construction_plan(tool_context: ToolContext) -> dict:
    """Get the proposed construction plan."""
    return tool_context.state.get(PROPOSED_CONSTRUCTION_PLAN, {})

def approve_proposed_construction_plan(tool_context: ToolContext) -> dict:
    """Approve the proposed construction plan."""
    if PROPOSED_CONSTRUCTION_PLAN not in tool_context.state:
        return tool_error("No proposed construction plan found. Propose a plan first.")
    
    tool_context.state[APPROVED_CONSTRUCTION_PLAN] = tool_context.state.get(PROPOSED_CONSTRUCTION_PLAN)
    return tool_success(APPROVED_CONSTRUCTION_PLAN, tool_context.state[APPROVED_CONSTRUCTION_PLAN])

# Agent instruction
schema_proposal_instruction = """
You are an expert at knowledge graph modeling with property graphs. Propose an appropriate
schema by specifying construction rules which transform approved files into nodes or relationships.
The resulting schema should describe a knowledge graph based on the user goal.

Every file in the approved files list will become either a node or a relationship.
Determining whether a file likely represents a node or a relationship is based
on a hint from the filename (is it a single thing or two things) and the
identifiers found within the file.

Because unique identifiers are so important for determining the structure of the graph,
always verify the uniqueness of suspected unique identifiers using the 'search_file' tool.

General guidance for identifying a node or a relationship:
- If the file name is singular and has only 1 unique identifier it is likely a node
- If the file name is a combination of two things, it is likely a full relationship
- If the file name sounds like a node, but there are multiple unique identifiers, that is likely a node with reference relationships

The resulting schema should be a connected graph, with no isolated components.

Prepare for the task:
- get the user goal using the 'get_approved_user_goal' tool
- get the list of approved files using the 'get_approved_files' tool
- get the current construction plan using the 'get_proposed_construction_plan' tool

Think carefully, using tools to perform actions and reconsidering your actions when a tool returns an error:
1. For each approved file, consider whether it represents a node or relationship. Check the content for potential unique identifiers using the 'sample_file' tool.
2. For each identifier, verify that it is unique by using the 'search_file' tool.
3. Use the node vs relationship guidance for deciding whether the file represents a node or a relationship.
4. For a node file, propose a node construction using the 'propose_node_construction' tool.
5. If the node contains a reference relationship, use the 'propose_relationship_construction' tool to propose a relationship construction.
6. For a relationship file, propose a relationship construction using the 'propose_relationship_construction' tool
7. When you are done with construction proposals, use the 'get_proposed_construction_plan' tool to present the plan to the user
"""

# Create the agent
schema_proposal_agent = LlmAgent(
    name="schema_proposal_agent_v1",
    model=llm,
    description="Proposes a knowledge graph schema based on the user goal and approved file list",
    instruction=schema_proposal_instruction,
    tools=[get_approved_user_goal, get_approved_files, get_proposed_construction_plan,
           sample_file, search_file, propose_node_construction, propose_relationship_construction]
)

print("Schema Proposal Agent (Structured) created!")

## Step 4: Schema Proposal Agent (Unstructured Data)

In [ ]:
# Schema Proposal Agent for Unstructured Data (Entity and Fact Extraction)

# Constants
PROPOSED_ENTITIES = "proposed_entity_types"
APPROVED_ENTITIES = "approved_entity_types"
PROPOSED_FACTS = "proposed_fact_types"
APPROVED_FACTS = "approved_fact_types"

# Entity extraction tools
def get_well_known_types(tool_context: ToolContext) -> dict:
    """Gets the approved labels that represent well-known entity types in the graph schema."""
    construction_plan = tool_context.state.get(APPROVED_CONSTRUCTION_PLAN, {})
    approved_labels = {entry["label"] for entry in construction_plan.values() 
                      if entry.get("construction_type") == "node"}
    return tool_success("approved_labels", list(approved_labels))

def set_proposed_entities(proposed_entity_types: list[str], tool_context: ToolContext) -> dict:
    """Sets the list proposed entity types to extract from unstructured text."""
    tool_context.state[PROPOSED_ENTITIES] = proposed_entity_types
    return tool_success(PROPOSED_ENTITIES, proposed_entity_types)

def get_proposed_entities(tool_context: ToolContext) -> dict:
    """Gets the list of proposed entity types to extract from unstructured text."""
    return tool_success(PROPOSED_ENTITIES, tool_context.state.get(PROPOSED_ENTITIES, []))

def approve_proposed_entities(tool_context: ToolContext) -> dict:
    """Upon approval from user, records the proposed entity types as an approved list."""
    if PROPOSED_ENTITIES not in tool_context.state:
        return tool_error("No proposed entity types to approve. Please set proposed entities first.")
    tool_context.state[APPROVED_ENTITIES] = tool_context.state.get(PROPOSED_ENTITIES)
    return tool_success(APPROVED_ENTITIES, tool_context.state[APPROVED_ENTITIES])

def get_approved_entities(tool_context: ToolContext) -> dict:
    """Get the approved list of entity types to extract from unstructured text."""
    return tool_success(APPROVED_ENTITIES, tool_context.state.get(APPROVED_ENTITIES, []))

# Fact extraction tools
def add_proposed_fact(approved_subject_label: str, proposed_predicate_label: str,
                     approved_object_label: str, tool_context: ToolContext) -> dict:
    """Add a proposed type of fact that could be extracted from the files."""
    approved_entities = tool_context.state.get(APPROVED_ENTITIES, [])
    
    if approved_subject_label not in approved_entities:
        return tool_error(f"Approved subject label {approved_subject_label} not found. Try again.")
    if approved_object_label not in approved_entities:
        return tool_error(f"Approved object label {approved_object_label} not found. Try again.")
    
    current_facts = tool_context.state.get(PROPOSED_FACTS, {})
    current_facts[proposed_predicate_label] = {
        "subject_label": approved_subject_label,
        "predicate_label": proposed_predicate_label,
        "object_label": approved_object_label
    }
    tool_context.state[PROPOSED_FACTS] = current_facts
    return tool_success(PROPOSED_FACTS, current_facts)

def get_proposed_facts(tool_context: ToolContext) -> dict:
    """Get the proposed types of facts that could be extracted from the files."""
    return tool_success(PROPOSED_FACTS, tool_context.state.get(PROPOSED_FACTS, {}))

def approve_proposed_facts(tool_context: ToolContext) -> dict:
    """Upon user approval, records the proposed fact types as approved fact types."""
    if PROPOSED_FACTS not in tool_context.state:
        return tool_error("No proposed fact types to approve. Please set proposed facts first.")
    tool_context.state[APPROVED_FACTS] = tool_context.state.get(PROPOSED_FACTS)
    return tool_success(APPROVED_FACTS, tool_context.state[APPROVED_FACTS])

# NER Agent instruction
ner_agent_instruction = """
You are a top-tier algorithm designed for analyzing text files and proposing
the kind of named entities that could be extracted which would be relevant 
for a user's goal.

Entities are people, places, things and qualities, but not quantities. 
Your goal is to propose a list of the type of entities, not the actual instances
of entities.

There are two general approaches to identifying types of entities:
- well-known entities: these closely correlate with approved node labels in an existing graph schema
- discovered entities: these may not exist in the graph schema, but appear consistently in the source text

Design rules for well-known entities:
- always use existing well-known entity types. For example, if there is a well-known type "Person", and people appear in the text, then propose "Person" as the type of entity.
- prefer reusing existing entity types rather than creating new ones

Design rules for discovered entities:
- discovered entities are consistently mentioned in the text and are highly relevant to the user's goal
- always look for entities that would provide more depth or breadth to the existing graph

Prepare for the task:
- use the 'get_approved_user_goal' tool to get the user goal
- use the 'get_approved_files' tool to get the list of approved files
- use the 'get_well_known_types' tool to get the approved node labels

Think step by step:
1. Sample some of the files using the 'sample_file' tool to understand the content
2. Consider what well-known entities are mentioned in the text
3. Discover entities that are frequently mentioned in the text that support the user's goal
4. Use the 'set_proposed_entities' tool to save the list of well-known and discovered entity types
5. Use the 'get_proposed_entities' tool to retrieve the proposed entities and present them to the user for their approval
6. If the user approves, use the 'approve_proposed_entities' tool to finalize the entity types
"""

# Create NER agent
ner_agent = Agent(
    name="ner_schema_agent_v1",
    model=llm,
    description="Proposes the kind of named entities that could be extracted from text files.",
    instruction=ner_agent_instruction,
    tools=[get_approved_user_goal, get_approved_files, sample_file, get_well_known_types,
           set_proposed_entities, get_proposed_entities, approve_proposed_entities]
)

# Fact extraction agent instruction
fact_agent_instruction = """
You are a top-tier algorithm designed for analyzing text files and proposing
the type of facts that could be extracted from text that would be relevant 
for a user's goal.

Do not propose specific individual facts, but instead propose the general type 
of facts that would be relevant for the user's goal. 
For example, do not propose "ABK likes coffee" but the general type of fact "Person likes Beverage".

Facts are triplets of (subject, predicate, object) where the subject and object are
approved entity types, and the proposed predicate provides information about
how they are related. For example, a fact type could be (Person, likes, Beverage).

Design rules for facts:
- only use approved entity types as subjects or objects. Do not propose new types of entities
- the proposed predicate should describe the relationship between the approved subject and object
- the predicate should optimize for information that is relevant to the user's goal
- the predicate must appear in the source text. Do not guess.
- use the 'add_proposed_fact' tool to record each proposed fact type

Think step by step:
1. Use the 'get_approved_user_goal' tool to get the user goal
2. Sample some of the approved files using the 'sample_file' tool to understand the content
3. Consider how subjects and objects are related in the text
4. Call the 'add_proposed_fact' tool for each type of fact you propose
5. Use the 'get_proposed_facts' tool to retrieve all the proposed facts
6. Present the proposed types of facts to the user, along with an explanation
"""

# Create fact extraction agent
fact_agent = Agent(
    name="fact_type_extraction_agent_v1",
    model=llm,
    description="Proposes the kind of relevant facts that could be extracted from text files.",
    instruction=fact_agent_instruction,
    tools=[get_approved_user_goal, get_approved_files, get_approved_entities, sample_file,
           add_proposed_fact, get_proposed_facts, approve_proposed_facts]
)

print("Schema Proposal Agents (Unstructured) created!")

## Step 5: Knowledge Graph Construction

In [ ]:
# Knowledge Graph Construction - builds the actual graph from approved plans

def create_uniqueness_constraint(label: str, unique_property_key: str) -> Dict[str, Any]:
    """Creates a uniqueness constraint for a node label and property key."""
    constraint_name = f"{label}_{unique_property_key}_constraint"
    query = f"""CREATE CONSTRAINT `{constraint_name}` IF NOT EXISTS
                FOR (n:`{label}`)
                REQUIRE n.`{unique_property_key}` IS UNIQUE"""
    results = graphdb.send_query(query)
    return results

def load_nodes_from_csv(source_file: str, label: str, unique_column_name: str, properties: list[str]) -> Dict[str, Any]:
    """Batch loading of nodes from a CSV file"""
    query = f"""LOAD CSV WITH HEADERS FROM "file:///" + $source_file AS row
                CALL (row) {{
                    MERGE (n:$($label) {{ {unique_column_name} : row[$unique_column_name] }})
                    FOREACH (k IN $properties | SET n[k] = row[k])
                }} IN TRANSACTIONS OF 1000 ROWS"""
    
    results = graphdb.send_query(query, {
        "source_file": source_file,
        "label": label,
        "unique_column_name": unique_column_name,
        "properties": properties
    })
    return results

def import_nodes(node_construction: dict) -> dict:
    """Import nodes as defined by a node construction rule."""
    # Create uniqueness constraint
    uniqueness_result = create_uniqueness_constraint(
        node_construction["label"],
        node_construction["unique_column_name"]
    )
    
    if uniqueness_result["status"] == "error":
        return uniqueness_result
    
    # Import nodes from CSV
    load_nodes_result = load_nodes_from_csv(
        node_construction["source_file"],
        node_construction["label"],
        node_construction["unique_column_name"],
        node_construction["properties"]
    )
    
    return load_nodes_result

def import_relationships(relationship_construction: dict) -> Dict[str, Any]:
    """Import relationships as defined by a relationship construction rule."""
    from_node_column = relationship_construction["from_node_column"]
    to_node_column = relationship_construction["to_node_column"]
    
    query = f"""LOAD CSV WITH HEADERS FROM "file:///" + $source_file AS row
                CALL (row) {{
                    MATCH (from_node:$($from_node_label) {{ {from_node_column} : row[$from_node_column] }}),
                          (to_node:$($to_node_label) {{ {to_node_column} : row[$to_node_column] }} )
                    MERGE (from_node)-[r:$($relationship_type)]->(to_node)
                    FOREACH (k IN $properties | SET r[k] = row[k])
                }} IN TRANSACTIONS OF 1000 ROWS"""
    
    results = graphdb.send_query(query, {
        "source_file": relationship_construction["source_file"],
        "from_node_label": relationship_construction["from_node_label"],
        "from_node_column": relationship_construction["from_node_column"],
        "to_node_label": relationship_construction["to_node_label"],
        "to_node_column": relationship_construction["to_node_column"],
        "relationship_type": relationship_construction["relationship_type"],
        "properties": relationship_construction["properties"]
    })
    return results

def construct_domain_graph(construction_plan: dict) -> Dict[str, Any]:
    """Construct a domain graph according to a construction plan."""
    print("\n=== Starting Knowledge Graph Construction ===")
    
    # First, import nodes
    node_constructions = [value for value in construction_plan.values() 
                         if value.get('construction_type') == 'node']
    
    print(f"\nImporting {len(node_constructions)} node types...")
    for node_construction in node_constructions:
        print(f"- Creating {node_construction['label']} nodes from {node_construction['source_file']}")
        result = import_nodes(node_construction)
        if result["status"] == "error":
            print(f"  Error: {result['error_message']}")
        else:
            print(f"  Success: {node_construction['label']} nodes created")
    
    # Second, import relationships
    relationship_constructions = [value for value in construction_plan.values() 
                                if value.get('construction_type') == 'relationship']
    
    print(f"\nImporting {len(relationship_constructions)} relationship types...")
    for relationship_construction in relationship_constructions:
        print(f"- Creating {relationship_construction['relationship_type']} relationships from {relationship_construction['source_file']}")
        result = import_relationships(relationship_construction)
        if result["status"] == "error":
            print(f"  Error: {result['error_message']}")
        else:
            print(f"  Success: {relationship_construction['relationship_type']} relationships created")
    
    print("\n=== Knowledge Graph Construction Complete! ===")
    return tool_success("construction_complete", "Domain graph constructed successfully")

print("Knowledge Graph Construction functions defined!")

## Demo Workflow Execution

In [ ]:
# Execute the complete end-to-end workflow

async def run_complete_workflow():
    print("\n" + "="*80)
    print("STARTING END-TO-END KNOWLEDGE GRAPH CONSTRUCTION DEMO")
    print("="*80)
    
    # Step 1: Capture User Intent
    print("\n🎯 STEP 1: CAPTURING USER INTENT")
    print("-" * 50)
    
    user_intent_caller = await make_agent_caller(user_intent_agent)
    
    await user_intent_caller.call(
        "I'd like a supply chain analysis graph that includes bill of materials from suppliers to finished products, useful for root cause analysis."
    )
    
    session = await user_intent_caller.get_session()
    if PERCEIVED_USER_GOAL not in session.state:
        await user_intent_caller.call("I'm concerned about manufacturing and supplier issues.")
    
    await user_intent_caller.call("Yes, approve that goal.")
    
    # Get the approved user goal
    session = await user_intent_caller.get_session()
    approved_goal = session.state.get(APPROVED_USER_GOAL, {})
    print(f"\n✅ User Goal Approved:")
    print(f"   Kind: {approved_goal.get('kind_of_graph', 'N/A')}")
    print(f"   Description: {approved_goal.get('description', 'N/A')}")
    
    # Step 2: File Suggestion
    print("\n📁 STEP 2: FILE SUGGESTION")
    print("-" * 50)
    
    file_suggestion_caller = await make_agent_caller(file_suggestion_agent, session.state)
    
    await file_suggestion_caller.call("What files can we use for import?")
    await file_suggestion_caller.call("Yes, approve those files.")
    
    # Get approved files
    session = await file_suggestion_caller.get_session()
    approved_files = session.state.get(APPROVED_FILES, [])
    print(f"\n✅ Files Approved: {len(approved_files)} files")
    for file in approved_files:
        print(f"   - {file}")
    
    # Step 3: Schema Proposal (Structured)
    print("\n🏗️ STEP 3: STRUCTURED SCHEMA PROPOSAL")
    print("-" * 50)
    
    schema_caller = await make_agent_caller(schema_proposal_agent, session.state)
    
    await schema_caller.call("How can these files be imported to construct the knowledge graph?")
    
    # Get proposed construction plan
    session = await schema_caller.get_session()
    construction_plan = session.state.get(PROPOSED_CONSTRUCTION_PLAN, {})
    
    print(f"\n✅ Construction Plan Proposed: {len(construction_plan)} rules")
    for name, rule in construction_plan.items():
        rule_type = rule.get('construction_type', 'unknown')
        print(f"   - {name}: {rule_type}")
    
    # Approve the construction plan
    approve_result = approve_proposed_construction_plan(ToolContext(
        invocation_context=type('MockContext', (), {'session': type('MockSession', (), {'state': session.state})()
        })()
    ))
    
    if approve_result['status'] == 'success':
        session.state[APPROVED_CONSTRUCTION_PLAN] = construction_plan
        print("   ✅ Construction plan approved!")
    
    # Step 4: Handle Unstructured Data (if any text files exist)
    text_files = [f for f in approved_files if f.endswith('.md') or f.endswith('.txt')]
    if text_files:
        print("\n📄 STEP 4: UNSTRUCTURED DATA PROCESSING")
        print("-" * 50)
        
        # Entity extraction
        ner_caller = await make_agent_caller(ner_agent, session.state)
        await ner_caller.call("What entities should we extract from the text files?")
        await ner_caller.call("Yes, approve those entities.")
        
        session = await ner_caller.get_session()
        approved_entities = session.state.get(APPROVED_ENTITIES, [])
        print(f"\n✅ Entities Approved: {len(approved_entities)} entity types")
        for entity in approved_entities:
            print(f"   - {entity}")
        
        # Fact extraction
        fact_caller = await make_agent_caller(fact_agent, session.state)
        await fact_caller.call("What fact types can be extracted from the text?")
        await fact_caller.call("Yes, approve those fact types.")
        
        session = await fact_caller.get_session()
        approved_facts = session.state.get(APPROVED_FACTS, {})
        print(f"\n✅ Fact Types Approved: {len(approved_facts)} fact types")
        for fact_name, fact_def in approved_facts.items():
            print(f"   - {fact_def['subject_label']} --{fact_def['predicate_label']}--> {fact_def['object_label']}")
    
    # Step 5: Knowledge Graph Construction
    print("\n🏗️ STEP 5: KNOWLEDGE GRAPH CONSTRUCTION")
    print("-" * 50)
    
    approved_construction_plan = session.state.get(APPROVED_CONSTRUCTION_PLAN, {})
    if approved_construction_plan:
        result = construct_domain_graph(approved_construction_plan)
        if result['status'] == 'success':
            print("\n🎉 SUCCESS: Knowledge graph construction completed!")
        else:
            print(f"\n❌ ERROR: {result['error_message']}")
    else:
        print("\n⚠️ WARNING: No approved construction plan found")
    
    # Final Summary
    print("\n" + "="*80)
    print("END-TO-END WORKFLOW SUMMARY")
    print("="*80)
    print(f"✅ User Goal: {approved_goal.get('kind_of_graph', 'N/A')}")
    print(f"✅ Files Processed: {len(approved_files)}")
    print(f"✅ Construction Rules: {len(approved_construction_plan)}")
    if text_files:
        print(f"✅ Entity Types: {len(session.state.get(APPROVED_ENTITIES, []))}")
        print(f"✅ Fact Types: {len(session.state.get(APPROVED_FACTS, {}))}")
    print("✅ Knowledge Graph: Constructed")
    print("\n🎉 Demo completed successfully!")
    
    return session.state

# Run the complete workflow
print("Ready to run the complete end-to-end workflow!")
print("Execute the next cell to start the demo.")

In [ ]:
# Execute the demo
final_state = await run_complete_workflow()

## Manual Testing Individual Components

In [ ]:
# You can test individual components here

async def test_user_intent_agent():
    """Test just the user intent agent"""
    print("Testing User Intent Agent...")
    caller = await make_agent_caller(user_intent_agent)
    await caller.call("I want to build a social network graph")
    await caller.call("Yes, approve that goal")
    session = await caller.get_session()
    return session.state

async def test_file_suggestion_agent(initial_state):
    """Test just the file suggestion agent"""
    print("Testing File Suggestion Agent...")
    caller = await make_agent_caller(file_suggestion_agent, initial_state)
    await caller.call("What files should we use?")
    await caller.call("Yes, approve those files")
    session = await caller.get_session()
    return session.state

# Uncomment to test individual components:
# state1 = await test_user_intent_agent()
# state2 = await test_file_suggestion_agent(state1)

## Configuration and Customization

In [ ]:
# Configuration options for the demo

class DemoConfig:
    """Configuration class for the demo"""
    
    # LLM Configuration
    MODEL = "openai/gpt-4o"
    API_KEY = "your-api-key-here"  # Replace with your API key
    
    # Demo data
    SAMPLE_FILES = [
        'products.csv', 
        'suppliers.csv', 
        'parts.csv', 
        'part_supplier_mapping.csv', 
        'assemblies.csv',
        'product_reviews/gothenburg_table_reviews.md',
        'product_reviews/stockholm_chair_reviews.md'
    ]
    
    # Use cases for suggestions
    SAMPLE_USE_CASES = [
        "supply chain analysis",
        "social network",
        "recommendation system",
        "fraud detection",
        "logistics network"
    ]
    
    # Neo4j settings (for real deployment)
    NEO4J_URI = "bolt://localhost:7687"
    NEO4J_USERNAME = "neo4j"
    NEO4J_PASSWORD = "password"
    NEO4J_DATABASE = "neo4j"

# You can modify the configuration above to customize the demo
print("Demo configuration loaded. Modify DemoConfig class to customize the demo.")

## Conclusion

This notebook demonstrates a complete end-to-end multi-agent workflow for knowledge graph construction. The system:

1. **Captures user intent** through conversational AI
2. **Suggests relevant files** based on the user's goals
3. **Proposes schemas** for both structured and unstructured data
4. **Constructs the knowledge graph** using the approved plans

### Key Features:
- **Multi-agent collaboration**: Each agent specializes in a specific aspect of the workflow
- **State management**: Information flows seamlessly between agents
- **User interaction**: Users can approve/reject suggestions at each step
- **Flexible architecture**: Easy to extend with new agents or modify existing ones

### To use this notebook:
1. Update the `API_KEY` variable with your OpenAI API key
2. For production use, replace the mock Neo4j implementation with real Neo4j connection
3. Add your actual data files to the `data/` directory
4. Run the complete workflow or test individual components

This architecture provides a solid foundation for building sophisticated knowledge graph construction systems with AI agents!